In [1]:
!pip install -q -U tensorflow-text

In [2]:
!pip install -q tf-models-official

In [3]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

C:\Users\adars\anaconda3\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [4]:
import pandas as pd
import numpy as np

col_names = ['PID','Text_data','Label']
train_df = pd.read_csv("train.tsv", header=0, sep='\t', names=col_names)
dev_df = pd.read_csv("dev_with_labels.tsv", header=0, sep='\t', names=col_names)
df_combined = pd.concat([train_df, dev_df])
df_combined = df_combined.drop_duplicates(subset=['Text_data'])
df_combined

,PID,Text_data,Label
0,train_pid_1,Waiting for my mind to have a breakdown once t...,moderate
1,train_pid_2,My new years resolution : I'm gonna get my ass...,moderate
2,train_pid_3,New year : Somone else Feeling like 2020 will ...,moderate
3,train_pid_4,"My story I guess : Hi, Im from Germany and my ...",moderate
4,train_pid_5,Sat in the dark and cried myself going into th...,moderate
...,...,...,...
4491,dev_pid_4492,Aren’t we all just tired? : I’ve been depresse...,severe
4492,dev_pid_4493,NEED HELP COPING : I had my life pretty much f...,severe
4493,dev_pid_4494,Qutting Zoloft Cold Turkey : I was on 75 mg se...,severe
4494,dev_pid_4495,Crying : I’m coming off my antidepressants and...,severe


In [5]:
# def reLabel(label):
#     if label == 'not depression':
#         return 0
#     if label == 'moderate':
#         return 1
#     if label == 'severe':
#         return 2

# df_combined['Label'] = df_combined['Label'].apply(reLabel)
# print(df_combined)
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(df_combined['Label'])
df_combined['Label'] = encoder.transform(df_combined['Label'])
df_combined

label_tf = (tf.one_hot(df_combined['Label'], depth = 3))
label_tf

<tf.Tensor: shape=(7006, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)>

In [6]:
encoder = LabelBinarizer()
encoder.fit(df_combined['Label'])
encoder.transform(df_combined['Label'])

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [7]:
lengths = []
for i in df_combined['Text_data']:
    lengths.append(len(i))
print(np.average(lengths))
print(np.median(lengths))

845.0632315158435
509.0


In [8]:
from stop_words import get_stop_words
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

stop_words = list(get_stop_words('en'))         #About 900 stopwords
nltk_words = list(stopwords.words('english')) #About 150 stopwords
stop_words.extend(nltk_words)

filtered_sentence = []

for text in df_combined['Text_data']:
    text =  ' '.join([word for word in text.split() if word not in stop_words])
    filtered_sentence.append(text)

df_combined['Text_data'] = filtered_sentence
lengths = []
for i in df_combined['Text_data']:
    lengths.append(len(i))
print(np.average(lengths))
print(np.median(lengths))

train_ds, dev_ds, train_labels, dev_labels = train_test_split(df_combined['Text_data'], df_combined['Label'], test_size = 0.2)

572.3930916357408
349.0


In [9]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(df_combined['Label']),
                                        y = df_combined['Label']                                                    
                                    )
class_weights = dict(zip(np.unique(df_combined['Label']), class_weights)),
class_weights = class_weights[0]
print(class_weights)

{0: 0.6467275916182036, 1: 0.8795982423101067, 2: 3.155855855855856}


In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer

train_text = train_ds.to_numpy()
# tok = Tokenizer(oov_token='<unk>')
# tok.fit_on_texts(train_text)
# tok.word_index['<pad>'] = 0
# tok.index_word[0] = '<pad>'

#train_seqs = tok.texts_to_sequences(train_text)
#train_seqs = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

train_labels = tf.one_hot(train_labels, depth = 3)

valid_text = dev_ds.to_numpy()
#valid_seqs = tok.texts_to_sequences(valid_text)
#valid_seqs = tf.keras.preprocessing.sequence.pad_sequences(valid_seqs, padding='post')

valid_labels = tf.one_hot(dev_labels, depth = 3)

class_names = {0:'not depression',1:'moderate',2:'severe'}

# CONVERT TO TF DATASETS

train_ds = tf.data.Dataset.from_tensor_slices((train_text,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((valid_text,valid_labels))

train_ds = train_ds.shuffle(100).batch(32)
valid_ds = valid_ds.batch(32)

#PREFETCH

train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
valid_ds = valid_ds.prefetch(buffer_size=tf.data.AUTOTUNE)



In [11]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(3):
    print(f'Review: {text_batch.numpy()[i]}')
    label = label_batch
    print(f'Label : {label[i]}')

Review: b"I'm jealous happy people : I wonder like trust family surrounded friends loved ones. I wonder like mind free violent suicidal thoughts. Do people understand lucky are? I wish I wake without feeling dreadful, I wish I simple things interactions without crippling anxiety. 2020 shitty start far"
Label : [1. 0. 0.]
Review: b'A friend? : Anyone bored also lonely. When get lonely go anywhere find someone talk online like texting something. That\xe2\x80\x99s right now. It\xe2\x80\x99s stupid idk. I\xe2\x80\x99m stupid idiot right now. So ignore me. Probs will. I\xe2\x80\x99m good ignored. Sorry. I\xe2\x80\x99m really stupid annoying.'
Label : [1. 0. 0.]
Review: b'Does anyone else struggle concept self love? I feel like struggle may make hard get better. : [removed]'
Label : [0. 1. 0.]


In [12]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [13]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [14]:
text_test = ['I am so sad and depressed right now']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_mask', 'input_word_ids', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [  101  1045  2572  2061  6517  1998 14777  2157  2085   102     0     0]
Input Mask : [1 1 1 1 1 1 1 1 1 1 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [15]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [16]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[-0.2610633   0.96756256  0.14548907  0.5520671   0.54360384  0.3331888
  0.9933292  -0.9728479   0.00306453 -0.99994177  0.38466614 -0.9581365 ]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[ 0.22679493  0.44570273  1.0670617  ... -2.8421602  -0.40608317
   0.7170032 ]
 [ 0.2935018   0.34215915  0.1386933  ... -2.0460887  -0.6787251
  -0.04941279]
 [-0.14717127  0.5424799  -0.0448623  ... -1.6283281  -1.3944806
   0.18597376]
 ...
 [ 0.40284657  0.3325398   1.359091   ... -2.279018   -0.22745772
   0.5909338 ]
 [ 0.32569647 -0.31818882  0.00349967 ... -0.8752294   0.6427677
   0.6850786 ]
 [-0.08367953 -0.42930108 -0.08188547 ... -0.8535627   0.5253165
   0.54266447]]


In [17]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(3, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [18]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor([[0.41253728 0.3949361  0.7011579 ]], shape=(1, 3), dtype=float32)


In [19]:
import pydot
tf.keras.utils.plot_model(classifier_model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [20]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = tf.metrics.CategoricalAccuracy()

In [21]:
for i in train_ds.take(1):
    print(i)

(<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b"I absolutely nothing left live for. : I capable making changes I need move forward, around seek bury further. My mom stole savings ($6k me, maybe $500 her) I car. Without car, I work, (extremely limited public transport area) without job I afford medicine keep functional. I incredibly debilitating ADHD insurance, I human treatment. Just wasteful leech. I use heavy amounts marijuana cope inability live truly help. I friends, mom worst family. I bruised scarred abuse I taken years, I go down. This coronavirus thing gotten thinking, maybe I try get it. Try die, then. It's I want. It's I need.",
       b'Humans : Asked help reddit end getting shat on. I feel like happened constantly work real life. Thought I turn Reddit sometimes, feels too. Suicidal af really trying best treat everyone nice. Sad part is, others ever treat like trash without respect. I\xe2\x80\x99m trying kind, hell people darn mean',
       b'Should known : Just some

In [22]:
epochs = 10
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [23]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [24]:
classifier_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [25]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                               validation_data=valid_ds,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/10
176/176 [==============================] - 74s 393ms/step - loss: 0.9352 - categorical_accuracy: 0.5614 - val_loss: 0.9015 - val_categorical_accuracy: 0.6006
Epoch 2/10
176/176 [==============================] - 69s 391ms/step - loss: 0.8191 - categorical_accuracy: 0.6135 - val_loss: 0.8270 - val_categorical_accuracy: 0.6127
Epoch 3/10
176/176 [==============================] - 69s 392ms/step - loss: 0.7452 - categorical_accuracy: 0.6563 - val_loss: 0.8051 - val_categorical_accuracy: 0.6234
Epoch 4/10
176/176 [==============================] - 69s 395ms/step - loss: 0.6782 - categorical_accuracy: 0.6975 - val_loss: 0.8396 - val_categorical_accuracy: 0.6113
Epoch 5/10
176/176 [==============================] - 69s 395ms/step - loss: 0.6024 - categorical_accuracy: 0.7323 - val_loss: 0.9199 - val_categorical_accuracy: 0.6191
Epoch 6/10
176/176 [==============================] - 69s 391ms

In [26]:
classifier_model.save('saved_model/BertCategorical10epochs')

In [27]:
test_df = pd.read_csv("test_data.tsv", header=0, sep='\t', names = ['pid', 'Text_data'])

filtered_sentence = []

for text in test_df['Text_data']:
    text =  ' '.join([word for word in text.split() if word not in stop_words])
    filtered_sentence.append(text)

test_df['Text_data'] = filtered_sentence

In [ ]:
results = tf.sigmoid(classifier_model(tf.constant(test_df['Text_data'])))

In [ ]:
results

In [ ]:
class_label = []

for i in results.numpy():
    class_label.append(np.argmax(i))

In [ ]:
test_df['class_label'] = class_label
test_df['class_label'] = test_df['class_label'].apply(reLabel)
print(test_df)

In [ ]:
test_df = test_df.drop('Text_data', axis=1)
test_df.to_csv('predictionsBERT10epochs.tsv', sep="\t")